# Assignment 6

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

tf.random.set_seed(42) #set tf random seed

In [ ]:
#you should have change it to your data set directory
!unzip /content/drive/MyDrive/dataset_emoji.zip

Archive:  /content/drive/MyDrive/dataset_emoji.zip
  inflating: test.csv                
  inflating: train.csv               


## Load and pre-proce the data

In [ ]:
def make_data(dir,tokinzer=None ,num_words = 10000, maxlen = None):
  data = pd.read_csv(dir)[['text', 'emoji']] #load the data set
  X_txt = data['text'] 
  y = data['emoji']
  num_of_classes = len(set(y)) # find the number of class 
  classes = dict(zip(set(y),range(num_of_classes)))
  rev_classes = dict(zip(range(num_of_classes), set(y)))
  y.replace(classes, inplace = True)
  if tokinzer==None: #if tokinzer = None new tokinzer will be defined  
                    #else the passed tokinzer will be used
    tokinzer = tf.keras.preprocessing.text.Tokenizer(num_words = num_words, oov_token='<oov>')
    tokinzer.fit_on_texts(X_txt)


  y_onehot = tf.one_hot(y, depth = num_of_classes)
  X_seq = tokinzer.texts_to_sequences(X_txt) # use tokinzer to convert the text into sequnece
  if maxlen == None: #if maxlen = None the maxlen will be set to the length of the text that has the maximum length
    X_seq = tf.keras.preprocessing.sequence.pad_sequences(X_seq)
    _, maxlen = X_seq.shape 
  else:   #else the passed maxlen will be used 
    X_seq = tf.keras.preprocessing.sequence.pad_sequences(X_seq, maxlen=maxlen)
    
  data_size, sq_size = X_seq.shape
  return X_seq, y_onehot, num_of_classes, rev_classes, tokinzer, data_size,sq_size ,num_words, maxlen



In [ ]:
X_train, y_train, num_of_classes, rev_classes, tokinzer, data_size,sq_size ,num_words, maxlen = make_data('/content/train.csv')
X_test, y_test, _,_,_,_,_,_,_ = make_data('/content/test.csv', tokinzer = tokinzer, maxlen=maxlen)

## define the model

In [ ]:
model = tf.keras.models.Sequential([                
                       Embedding(num_words + 1,64,input_length=sq_size)
                      ,LSTM(8)
                      ,Dense(num_of_classes, activation='softmax')
                      ])

In [ ]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics='acc')

In [ ]:
check_point = tf.keras.callbacks.ModelCheckpoint('mohammed_baqer.h5', monitor='val_acc')

In [8]:
model.fit(X_train, y_train, 
                   validation_data=(X_test, y_test), epochs=20, callbacks = [check_point])

Epoch 1/20
500/500 [==============================] - 14s 25ms/step - loss: 1.5710 - acc: 0.3867 - val_loss: 0.8884 - val_acc: 0.7060
Epoch 2/20
500/500 [==============================] - 12s 24ms/step - loss: 0.7572 - acc: 0.7472 - val_loss: 0.6569 - val_acc: 0.7740
Epoch 3/20
500/500 [==============================] - 12s 25ms/step - loss: 0.5217 - acc: 0.8448 - val_loss: 0.5356 - val_acc: 0.8280
Epoch 4/20
500/500 [==============================] - 12s 24ms/step - loss: 0.3516 - acc: 0.9000 - val_loss: 0.4413 - val_acc: 0.8590
Epoch 5/20
500/500 [==============================] - 12s 25ms/step - loss: 0.2305 - acc: 0.9321 - val_loss: 0.4041 - val_acc: 0.8710
Epoch 6/20
500/500 [==============================] - 12s 24ms/step - loss: 0.1851 - acc: 0.9403 - val_loss: 0.4038 - val_acc: 0.8760
Epoch 7/20
500/500 [==============================] - 13s 26ms/step - loss: 0.1371 - acc: 0.9596 - val_loss: 0.3991 - val_acc: 0.8795
Epoch 8/20
500/500 [==============================] - 13s 26ms

## Use the best model with colab form to predict a input text

### load the best model to use in the form

In [9]:
best_model = tf.keras.models.load_model('/content/mohammed_baqer.h5')
loss, acc = best_model.evaluate(X_test, y_test)
print('acc = ', acc, 'loss = ', loss)

63/63 [==============================] - 1s 6ms/step - loss: 0.5100 - acc: 0.8900
acc =  0.8899999856948853 loss =  0.5100082159042358


In [ ]:

def find_empji(text, tokinzer, maxlen, model = best_model):
  seq_txt = tokinzer.texts_to_sequences([text])
  seq_txt = tf.keras.preprocessing.sequence.pad_sequences(seq_txt, maxlen=maxlen)
  emoji = model.predict(seq_txt).argmax()
  return rev_classes[emoji]



In [ ]:
#@markdown ### use the model to predict the emoji for input text
input_thetext = 'you are lovely'  #@param {type: "string"}
print(find_empji(input_thetext, tokinzer, maxlen))



😍
